In [15]:
import numpy as np
from sklearn.metrics import confusion_matrix
import itertools

In [41]:
mpp_path = './data/mppcase2_test.npy'
bpnn_path = './data/bpnnPredicted.npy'
svm_path = './data/svm_rbf_pca_c_10.npy'
kmeans_path = './data/kmeans_test_results.npy'

labels = './data/truth_test.npy'

mpp = np.load(mpp_path)
svm = np.load(svm_path)
bpnn = np.load(bpnn_path)
kmeans = np.load(kmeans_path)

labels = np.load(labels)

In [42]:
def naive_bayes(all_results, labels):
    norm_conf_mats = []
    for i, results in enumerate(all_results):
        conf_mat = confusion_matrix(labels, results)
        norm_conf_mat = np.zeros(conf_mat.shape)
        for row in range(conf_mat.shape[0]):
            norm_conf_mat[row, :] = conf_mat[row, :] / np.sum(conf_mat[row, :])
        norm_conf_mats.append(norm_conf_mat)
    #print(np.array(norm_conf_mats)[:3, :])
    num_labels = len(np.unique(labels))
    num_clf = len(all_results)
    fused_labels = np.zeros(num_labels ** num_clf)
    clf_idxs = np.transpose([np.repeat([0, 1], 4), np.tile([0, 0, 1, 1], 2), np.tile([0, 1], 4)])
    for n in range(num_labels ** num_clf):
        probs = np.zeros(num_labels)
        for m in range(num_labels):
            probs[m] = norm_conf_mats[0][m, clf_idxs[n, 0]]
            for i in range(num_clf - 1):
                probs[m] *= norm_conf_mats[i + 1][m, clf_idxs[n, i + 1]]
        #print(probs)
        fused_labels[n] = np.argmax(probs)
    return fused_labels.astype(int)

In [43]:
all_clfs = np.array([mpp, svm, bpnn, kmeans])
all_clf_names = np.array(['mpp', 'svm', 'bpnn', 'kmeans'])
comb_idxs = [list(x) for x in itertools.combinations([0, 1, 2, 3], 3)]

for comb_idx in comb_idxs:
    clfs = all_clfs[comb_idx]
    clf_names = all_clf_names[comb_idx]
    fused_labels = naive_bayes(clfs, labels)
    fused_results = np.zeros(len(labels))
    mapping = np.transpose([np.repeat([0, 1], 4), np.tile([0, 0, 1, 1], 2), np.tile([0, 1], 4)])
    for n in range(len(labels)):
        label_set = np.array([clfs[0][n], clfs[1][n], clfs[2][n]])
        idx = np.where((mapping == label_set).all(axis=1))[0][0]
        fused_results[n] = fused_labels[idx]
    idx0 = fused_results == 0
    idx1 = fused_results == 1
    print(f'{clf_names[0]} + {clf_names[1]} + {clf_names[2]}:')
    print(f'overall acc: {round(np.mean(fused_results == labels), 4)}')
    print(f'no ship acc: {round(np.mean(fused_results[idx0] == labels[idx0]), 4)}')
    print(f'ship acc: {round(np.mean(fused_results[idx1] == labels[idx1]), 4)}\n')

mpp + svm + bpnn:
overall acc: 0.9733
no ship acc: 0.9865
ship acc: 0.9351

mpp + svm + kmeans:
overall acc: 0.9617
no ship acc: 0.9863
ship acc: 0.8944

mpp + bpnn + kmeans:
overall acc: 0.9283
no ship acc: 0.97
ship acc: 0.8204

svm + bpnn + kmeans:
overall acc: 0.96
no ship acc: 0.9863
ship acc: 0.8889

